In [1]:
import gzip
from correctionlib.schemav2 import CorrectionSet,Correction,Category,CategoryItem
from itertools import chain
import correctionlib.schemav2 as schema
from JSONTools import *
import pandas as pd
import numpy as np

Welcome to JupyROOT 6.24/00


In [2]:
from SandS_class import *

In [3]:
#years=["2017","2018","2016preVFP","2016postVFP"]
years=["2016postVFP"]
folders=["2016postVFP_UL"]
gain=[1.0,6.0,12.0]

In [4]:
for year,folder in zip(years,folders):
    print(year)
    corrs=[]
    runbinsall,fulldict=get_SS_runbins_and_SS_dict(year)
    workhorse=SandS(runbinsall,fulldict)
    corr = Correction.parse_obj(
        {
            "version": 2,
            "name": f'{year}_ScaleJSON',
            "description": "yes",
            "inputs": [
                {"name": "valtype","type": "string", "description": "correction (total_correction) or uncertainty (total_uncertainty)"},
                {"name": "gain","type": "int", "description": "seed gain"},
                {"name": "run","type": "real", "description": "run"},
                {"name": "eta","type": "real", "description": "supercluster eta"},
                {"name": "r9", "type": "real", "description": "r9"},
                {"name": "et", "type": "real", "description": "et"},],
                #{"name": "gain", "type": "real", "description": "gain"},
            "output": {"name": "weight", "type": "real", "description": "value"},
            "data": Category.parse_obj({
                "nodetype": "category",
                "input": "valtype",
                "content": [
                    schema.CategoryItem.parse_obj({
                        "key": valtype, 
                        "value": schema.Category.parse_obj({
                            "nodetype": "category",
                            "input": "gain",
                            "content":[CategoryItem.parse_obj({"key":gain,
                                                               "value":workhorse.getrunSSnew(runbinsall,gain,valtype)
                                                              }) for gain in [1,6,12] 
                            ]
                        })
                    })
                    for valtype in ["total_correction","total_uncertainty"]
                ]
            })
        })
    corrs.append(corr)


    #Save JSON
    cset = CorrectionSet(schema_version=2, corrections=corrs,description=f"These are the photon ID, Pixel veto and CSEV Scale Factors (nominal, up or down) for {year} Ultra Legacy dataset. Each kind of scale factors, depend on the chosen working point, some other quantities. More information is below! Please keep in the mind that if your analysis is sensitive to high pT photons (>500 GeV), please refer https://twiki.cern.ch/twiki/bin/view/CMS/EGMPhotonIDHighPtPhotons")
    with open(f"{folder}/SS.json", "w") as fout:
        fout.write(cset.json(exclude_unset=True, indent=4))

2016postVFP
https://raw.githubusercontent.com/cms-data/EgammaAnalysis-ElectronTools/master/ScalesSmearings/Run2016_UltraLegacy_postVFP_RunFineEtaR9Gain_scales.dat


In [5]:
from correctionlib import _core
#Download the correct JSON files
evaluator = _core.CorrectionSet.from_file('2016postVFP_UL/SS.json')

In [6]:
evaluator["2016postVFP_ScaleJSON"].evaluate("total_correction",1,284044.0,-2.5,0.5,100.0)

1.0180131299999997

In [7]:
evaluator["2016postVFP_ScaleJSON"].evaluate("total_uncertainty",1,284044.0,-2.5,0.5,100.0)

0.03054039389999999

In [8]:
from Scale_simplepython import *

In [9]:
getsimplepythonScale("scale_nominal", "2016_postVFP", 284044, 2.5, 0.45, 100, 1)

{13001}


1.0180131299999997

In [10]:
getsimplepythonScale("scale_uncertainty", "2016_postVFP", 284044, 2.5, 0.45, 100, 1)

{13001}


0.03054039389999999